In [1]:
%matplotlib inline

In [5]:
from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.dates as mdates
from matplotlib import rcParams

import numpy as np
import pandas as pd


import shapely.geometry as sgeom

import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader

import dask.dataframe as dd
import dask.bag as db
import dask.diagnostics as dg

import geopandas as gpd


paper_path = Path('../../paper/figures/')

slide_path = Path('../../slides/figures/intro/')
proposal_path = Path('../../draft/figures/intro/')
#rcParams['font.family'] = 'Segoe Print'

In [2]:
YEAR = 2021
url = f"https://noaa-ghcn-pds.s3.amazonaws.com/csv/by_year/{YEAR}.csv"
names = ['ID', 'DATE', 'ELEMENT', 'DATA_VALUE', 'M-FLAG', 'Q-FLAG', 'S-FLAG', 'OBS-TIME']
ds = dd.read_csv(url, storage_options={'anon':True},  names=names, memory_map=False, 
                  dtype={'DATA_VALUE':'object'}, parse_dates=['DATE', 'OBS-TIME'])

FileNotFoundError: An error occurred while calling the read_csv method registered to the pandas backend.
Original Message: https://noaa-ghcn-pds.s3.amazonaws.com/csv/by_year/2021.csv

In [4]:
YEAR = 2021
url = f"{YEAR}.csv"
names = ['ID', 'DATE', 'ELEMENT', 'DATA_VALUE', 'M-FLAG', 'Q-FLAG', 'S-FLAG', 'OBS-TIME']
ds = dd.read_csv(url, storage_options={'anon':True},  names=names, memory_map=False, 
                  dtype={'DATA_VALUE':'object'}, parse_dates=['DATE', 'OBS-TIME'])

C:\Users\story\miniconda3\envs\paper\Lib\site-packages\dask\dataframe\io\csv.py:632: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  head = reader(BytesIO(b_sample), nrows=sample_rows, **head_kwargs)


In [5]:
# {column name:extents of the fixed-width fields}
columns = {"ID": (0,11), "LATITUDE": (12, 20), "LONGITUDE": (21, 30), "ELEVATION": (31, 37),"STATE": (38, 40),
           "NAME": (41, 71), "GSN FLAG": (72, 75), "HCN/CRN FLAG": (76, 79),"WMO ID": (80, 85)}

In [6]:
df = pd.read_fwf("http://noaa-ghcn-pds.s3.amazonaws.com/ghcnd-stations.txt", 
                    colspecs=list(columns.values()), names=list(columns.keys())).dropna(subset=['STATE'])

In [7]:
df['STATE'].unique()

array(['AS', 'BH', 'BC', 'YT', 'NT', 'NU', 'AB', 'SK', 'MB', 'ON', 'QC',
       'NB', 'NS', 'PE', 'NL', 'MP', 'FM', 'GU', 'UM', 'PW', 'MH', 'PR',
       'SA', 'SD', 'CO', 'NE', 'AK', 'AL', 'AR', 'AZ', 'CA', 'TN', 'CT',
       'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY',
       'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND',
       'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
       'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY', 'VI'], dtype=object)

In [7]:
nydf = df[df['STATE'].str.match("NY")]

NameError: name 'df' is not defined

In [8]:
ds['ELEMENT'].unique().compute()

NameError: name 'ds' is not defined

In [9]:
nyds = ds[ds['ID'].isin(nydf['ID'].unique()) & ds['ELEMENT'].isin(['TAVG', 'PRCP' ])].compute()

NameError: name 'ds' is not defined

In [12]:
ny = nyds.merge(nydf, on='ID')

In [13]:
ny.columns

Index(['ID', 'DATE', 'ELEMENT', 'DATA_VALUE', 'M-FLAG', 'Q-FLAG', 'S-FLAG',
       'OBS-TIME', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'STATE', 'NAME',
       'GSN FLAG', 'HCN/CRN FLAG', 'WMO ID'],
      dtype='object')

In [14]:
ds['ELEMENT'].unique().compute()

C:\Users\story\miniconda3\envs\paper\Lib\site-packages\dask\dataframe\io\csv.py:195: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = reader(bio, **kwargs)
C:\Users\story\miniconda3\envs\paper\Lib\site-packages\dask\dataframe\io\csv.py:195: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = reader(bio, **kwargs)
C:\Users\story\miniconda3\envs\paper\Lib\site-packages\dask\dataframe\io\csv.py:195: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = reader(bio, **kwargs)
C:\Users\story\miniconda3\envs\paper\Lib\site-packages\dask\dataframe\io\csv.py:195: UserWarning: Could no

0     ELEMENT
1        TMAX
2        PRCP
3        TAVG
4        TMIN
       ...   
71       WT17
72       DASF
73       MDSF
74       WT10
75       WT18
Name: ELEMENT, Length: 76, dtype: object

In [15]:
nyd = ny.pivot(index=['NAME','LATITUDE', 'LONGITUDE', 'DATE'], columns=['ELEMENT'], values=['DATA_VALUE']).reset_index()

ValueError: Index contains duplicate entries, cannot reshape

In [ ]:
nyd

In [ ]:
nyd.columns = [c[0] if i <4 else c[1] for i, c in enumerate(nyd.columns.to_flat_index())]

In [ ]:
nyd.head()

In [ ]:
nyd['TAVGF'] =  (nyd['TAVG'].astype(float)/10) *(9/5) + 32

In [ ]:
nyd.head()

In [ ]:
nyd['PRCP'].astype(float).describe()

In [ ]:
nyd['PRCPI'] = nyd['PRCP'].astype(float)/10 * 0.039370 

In [ ]:
nyd['PRCPI'].describe()

In [ ]:
nyd.to_csv("nydata", index=False)

In [2]:
%conda install dask

error: incomplete escape \U at position 28